In [ ]:
# Make necessary imports

import numpy as np
import pandas as pd
import random

In [ ]:
# Expectation Maximization

from scipy.stats import multivariate_normal

def EM(X,K,iters):
    reg_cov = 1e-6*np.identity(len(X[0]))
    mu = np.random.randint(min(X[:,0]),max(X[:,0]),size=(K,len(X[0])))
    cov = np.zeros((K,len(X[0]),len(X[0])))
    for j in range(len(cov)):
        np.fill_diagonal(cov[j],5)
    weights = np.ones(K)/K

    for i in range(iters):
        # E step
        r_ic = np.zeros((len(X),len(cov)))

        for m,co,p,r in zip(mu,cov,weights,range(len(r_ic[0]))):
            co+=reg_cov
            mn = multivariate_normal(mean=m,cov=co)
            r_ic[:,r] = p*mn.pdf(X)/np.sum([weights_c*multivariate_normal(mean=mu_c,cov=cov_c).pdf(X) for weights_c,mu_c,cov_c in zip(weights,mu,cov+reg_cov)],axis=0)

        # M step
        mu = []
        cov = []
        weights = []

        for c in range(len(r_ic[0])):
            m_c = np.sum(r_ic[:,c],axis=0)
            mu_c = (1/m_c)*np.sum(X*r_ic[:,c].reshape(len(X),1),axis=0)
            mu.append(mu_c)
            cov.append(((1/m_c)*np.dot((np.array(r_ic[:,c]).reshape(len(X),1)*(X-mu_c)).T,(X-mu_c)))+reg_cov)
            weights.append(m_c/np.sum(r_ic))
    return mu

In [ ]:
# Kmeans++

def kmeansppinit(X,K):
    i = random.randint(0,X.shape[0])
    temp=np.array([X[i]])
    for k in range(1,K):
        D = np.array([]) 
        for x in X:
            D = np.append(D,np.min(np.sum((x-temp)**2)))
        prob = D/np.sum(D)
        probdash = np.cumsum(prob)
        r = random.random()
        i=0
        for j,p in enumerate(probdash):
            if r<p:
                i = j
                break
        temp = np.append(temp,[X[i]],axis=0)
        return temp

def kmeanspp(X,K,iters):
    centers = kmeansppinit(X,K)
    centers_new = np.zeros(centers.shape)
    clusters = np.zeros(X.shape[0])
    distances = np.zeros((X.shape[0],K))
    for i in range(iters):
        for k in range(K):
            distances[:,k] = np.linalg.norm(X - centers[k], axis=1)
        clusters = np.argmin(distances, axis = 1)
        for k in range(K):
            centers_new[k] = np.mean(X[clusters == k], axis=0)
    return centers_new

In [ ]:
# Generate GMM, data, and error measures

coin = [1,2]
cov = np.identity(32)

In [ ]:
mu1 = np.zeros(32)

In [ ]:
data = {}
em = {}
km = {}

In [ ]:
for c in [0.5,1,1.5,2,3,4,8]:
    data[c] = []
    mu2 = c * np.random.normal(0, 1) * np.ones(32)
    for i in range(10000):
        temp = random.choice(coin)
        if temp==1:
            data[c].append(np.random.multivariate_normal(mu1, cov))
        else:
            data[c].append(np.random.multivariate_normal(mu2, cov))
    data[c] = np.asarray(data[c])
    mu_em = EM(data[c],2,50)
    mu_km = kmeanspp(data[c],2,5)
    em[c] = np.linalg.norm(mu1 - mu_em[0]) + np.linalg.norm(mu2 - mu_em[1])
    km[c] = np.linalg.norm(mu1 - mu_km[0]) + np.linalg.norm(mu2 - mu_km[1])

In [ ]:
print('Data Generated from GMM')
data

Data Generated from GMM


{0.5: array([[-1.5220111 , -0.31869373,  1.09849653, ...,  0.42074836,
         -1.03236911, -0.57916896],
        [ 0.38143989,  1.38574458,  2.85544753, ...,  0.90685521,
          0.13100817, -0.95072418],
        [ 0.1172781 , -0.84014676, -0.80771889, ...,  0.46439028,
         -2.63039009, -1.35547545],
        ...,
        [ 0.57107379, -0.0569501 ,  1.64441245, ..., -0.76477313,
         -0.85899029,  0.03370256],
        [-1.17753562, -0.33684555, -0.45304029, ...,  0.99466845,
         -0.02738107,  0.02814964],
        [ 0.38890641,  0.21735419,  0.49917347, ...,  0.41984328,
          0.4177095 , -1.27315102]]),
 1: array([[ 0.7523828 ,  2.14734343,  0.7342221 , ..., -1.35355162,
         -0.98611815, -0.85655142],
        [-0.81747729,  0.77223428, -1.39597   , ..., -0.50084624,
         -0.40219913, -2.12801237],
        [-1.75701387, -0.92993717, -0.98453315, ..., -0.18537598,
         -0.20072995,  1.02789951],
        ...,
        [ 0.06579965,  0.89456518,  0.21296088

In [ ]:
print('Error Measures for Expectation Maximiziation')
pd.DataFrame({'c':list(em.keys()),'Error':list(em.values())})

Error Measures for Expectation Maximiziation


,c,Error
0,0.5,1.582734
1,1.0,2.226143
2,1.5,2.556495
3,2.0,5.886854
4,3.0,0.144796
5,4.0,0.765872
6,8.0,0.175057


In [ ]:
print('Error Measures for kmeans++')
pd.DataFrame({'c':list(km.keys()),'Error':list(km.values())})

Error Measures for Expectation Maximiziation


,c,Error
0,0.5,2.130648
1,1.0,2.745038
2,1.5,3.408102
3,2.0,1.612943
4,3.0,0.144796
5,4.0,6.275091
6,8.0,0.175057
